In [ ]:
pip install semantic-kernel

In [2]:
class WebSearchEngineSkill:
    """
    A search engine skill.
    """
    from semantic_kernel.orchestration.sk_context import SKContext
    from semantic_kernel.skill_definition import sk_function, sk_function_context_parameter

    def __init__(self, connector) -> None:
        self._connector = connector

    @sk_function(
        description="Performs a web search for a given query", name="searchAsync"
    )
    @sk_function_context_parameter(
        name="query",
        description="The search query",
    )
    async def search_async(self, query: str, context: SKContext) -> str:
        query = query or context.variables.get("query")[1]
        result = await self._connector.search_async(query, num_results=5, offset=0)
        return str(result)

In [3]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion, AzureChatCompletion
from semantic_kernel.planning.basic_planner import BasicPlanner
from semantic_kernel.connectors.search_engine import BingConnector
from dotenv import load_dotenv, find_dotenv

kernel = sk.Kernel()

deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
kernel.add_chat_service("dv", AzureChatCompletion(deployment, endpoint, api_key))

skills_directory = "../skills/"
summarize_skill = kernel.import_semantic_skill_from_directory(skills_directory, "MailSkill")
writer_skill = kernel.import_semantic_skill_from_directory(skills_directory, "TechSkill")

BING_API_KEY = sk.bing_search_settings_from_dot_env()
connector = BingConnector(BING_API_KEY)
kernel.import_skill(WebSearchEngineSkill(connector), skill_name="WebSearch")

ask = """
Write a mail to propose a presentation aboout Semantic kernel in business apps at a technical conference. The mail should be clear and concise. Use the web to get recent information on the topic."""


topic = """
Semantic Kernel in business apps"""

planner = BasicPlanner()
basic_plan = await planner.create_plan_async(ask, kernel)

print(basic_plan.generated_plan)

results = await planner.execute_plan_async(basic_plan, kernel)
print(results)

{
    "input": "Proposal for Semantic Kernel presentation at technical conference",
    "subtasks": [
        {"function": "WebSearch.searchAsync", "args": {"query": "Semantic Kernel in business applications"}},
        {"function": "MailSkill.MailGeneratorFunction"},
        {"function": "_GLOBAL_FUNCTIONS_.f_bf72dc49_a137_41bf_a055_b4e73577845f", "args": {"available_functions": ["WebSearch.searchAsync"], "goal": "Write a clear and concise email proposing a Semantic Kernel presentation at a technical conference using recent information from the web."}}
    ]
}
Dear [Recipient],

I am pleased to inform you that Semantic Kernel will be part of the Microsoft BUILD 2023 conference, which will take place from <strong>May 23rd to May 25th</strong>. We are excited to present at BUILD and showcase how Semantic Kernel can help you increase productivity for yourself and your team.

In the midst of the daily buzz about advanced AI, Microsoft is dedicating two sessions at next week's Build 2023 d